In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from datetime import datetime

from utils.preprocess import data_agg, time_normalization

import warnings
warnings.filterwarnings('ignore')

/home/staff/l/lipai.huang/.conda/envs/py38/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.7.1-CAPI-1.11.1 0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [2]:
raw_dir = os.path.join('data', 'raw')
processed_dir = os.path.join('data', 'processed')

# DB1
### Time-sensitive feature: activity_index

In [3]:
# find harris county
# 'date', 'lat', 'lng', 'act_idx'
db1 = pd.read_csv(os.path.join(raw_dir, '0201_0228_total_daily.csv'))
db1 = db1.loc[:, ~db1.columns.str.contains('^Unnamed')]
db1 = db1[['agg_day_period', 'xlon', 'xlat', 'activity_index_total']]
db1['time'] = time_normalization(db1['agg_day_period'])
db1.to_csv(os.path.join(processed_dir, 'activity.csv'), index=False)
db1

,agg_day_period,xlon,xlat,activity_index_total,time
0,2021-02-01,-95.411453,29.730396,0.027110,0.0
1,2021-02-01,-95.165634,29.819796,0.045658,0.0
2,2021-02-01,-95.138168,29.711314,0.031654,0.0
3,2021-02-01,-95.429306,30.029272,0.040890,0.0
4,2021-02-01,-95.378494,29.725626,0.077237,0.0
...,...,...,...,...,...
883689,2021-02-28,-95.684738,29.903163,0.032923,1.0
883690,2021-02-28,-95.699844,29.975755,0.054291,1.0
883691,2021-02-28,-95.714951,29.872206,0.009832,1.0
883692,2021-02-28,-95.708084,29.785237,0.122457,1.0


# DB2
### Time-sensitive feature: energy

In [4]:
db2 = pd.read_excel(os.path.join(raw_dir, 'filtered-Energy Sorted Zipcodes Louisiana.xlsx'))
db2.head()

,Zip Code,"Aug 29, 09:29 PM","Aug 30, 09:01 PM","Aug 31, 09:15 PM","Sep 01, 09:01 PM","Sep 02, 09:15 PM","Sep 03, 09:02 PM","Sep 04, 08:38 PM","Sep 05, 09:14 PM","Sep 06, 09:16 PM",...,"Sep 14, 09:17 PM","Sep 15, 08:52 PM","Sep 16, 08:14 PM","Sep 17, 08:19 PM","Sep 18, 11:51 PM","Sep 19, 09:48 PM","Sep 20, 10:07 PM","Sep 21, 11:06 PM","Sep 22, 09:18 PM","Sep 23, 12:14 PM"
0,70001,20478,20448,20448,20471,20301,16492,14377,14104,12339,...,5,4,1,4,3,0,3,5,2,2
1,70002,11032,11029,10935,10935,10935,11116,8499,7441,4575,...,14,26,3,1,15,8,16,0,0,41
2,70003,17253,17178,17179,17108,17108,17426,17406,17127,15766,...,16,27,20,19,12,5,13,127,8,10
3,70005,13375,13336,13336,13334,13334,10928,9333,4606,5741,...,4,3,3,3,2,1,3,2,4,9
4,70006,7723,7718,7718,7599,7599,7362,7318,7055,5463,...,5,5,3,1,3,0,0,0,1,1


In [5]:
db2_melt = db2.melt(id_vars=['Zip Code'], var_name='date', value_name='energy').rename(columns={'Zip Code': 'zip_code'})
db2_melt['date'] = db2_melt['date'].apply(lambda x: datetime.strptime(x, '%b %d, %I:%M %p'))
db2_melt = db2_melt.sort_values(by=['zip_code', 'date']).reset_index(drop=True)
db2 = db2_melt[['date', 'zip_code', 'energy']]
db2['time'] = time_normalization(db2['date'])
db2.to_csv(os.path.join(processed_dir, 'energy.csv'), index=False)
db2

,date,zip_code,energy,time
0,1900-08-29 21:29:00,70001,20478,0.000000
1,1900-08-30 21:01:00,70001,20448,0.039836
2,1900-08-31 21:15:00,70001,20448,0.080858
3,1900-09-01 21:01:00,70001,20471,0.121089
4,1900-09-02 21:15:00,70001,20301,0.162110
...,...,...,...,...
8133,1900-09-19 21:48:00,71483,0,0.853689
8134,1900-09-20 22:07:00,71483,0,0.894851
8135,1900-09-21 23:06:00,71483,0,0.937142
8136,1900-09-22 21:18:00,71483,0,0.974721


# DB3
### Time-sensitive feature: all the 4 features

In [4]:
db3_dir = os.path.join(raw_dir, 'CuebiqMicro_Harvey/Harvey_Micro/48201/average')
db3_sub_fs = os.listdir(db3_dir)
db3 = pd.DataFrame()
for f in db3_sub_fs:
    if not f.endswith('.csv'):
        continue
    db3_sub_f = pd.read_csv(os.path.join(db3_dir, f))
    db3_sub_f = db3_sub_f.loc[:, ~db3_sub_f.columns.str.contains('^Unnamed')]
    if db3.empty:
        db3 = db3_sub_f
    else:
        db3 = pd.concat([db3, db3_sub_f], axis=0)
db3 = db3.groupby(['FIPS'], as_index=False).agg(dict(zip(db3.columns[1:], ['mean']*len(db3.columns[1:]))))
db3.to_csv(os.path.join(processed_dir, 'CuebiqMicro_Harvey.csv'), index=False)
db3

,FIPS,CBG_Avg_Time(min),CBG_Avg_Distance(m),CBG_Avg_Trips,CBG_Avg_ROG(m)
0,48201100000,107.296802,4018.707364,3.156020,10113.662408
1,48201210100,104.213268,6103.488950,2.784135,12248.405482
2,48201210400,87.282331,3480.476643,3.137605,7580.504221
3,48201210500,88.823493,4141.978605,2.857448,8516.787420
4,48201210600,103.030483,5121.375618,2.453517,9403.280683
...,...,...,...,...,...
781,48201555701,119.785921,5548.585766,2.372287,10732.413942
782,48201555702,118.516782,6029.470310,2.378730,11195.082764
783,48201556000,101.165925,6667.235861,2.964854,16031.097532
784,48201980000,76.017172,3683.365172,2.773161,9937.521027


# DB4: HomeCBG to POI
### Time-sensitive feature: count

In [6]:
db4_dir = os.path.join(raw_dir, 'hurricane_ida/unzip/Ida_homecbg_poi')
db4 = data_agg(db4_dir)
db4['time'] = time_normalization(db4['date'])

100%|██████████| 61/61 [00:06<00:00,  9.89it/s]


In [7]:
db4.to_csv(os.path.join(processed_dir, 'Ida_homecbg_poi.csv'), index=False)

In [8]:
db4

,date,home_block_group_id,place_id,count,time
0,2021-08-01,10010201002,2738065,1,0.000000
1,2021-08-06,10010201002,9085723,1,0.083333
2,2021-08-06,10010201002,29167162,1,0.083333
3,2021-09-01,10010202001,2649100,1,0.516667
4,2021-09-03,10010202001,9094173,1,0.550000
...,...,...,...,...,...
7660162,2021-09-25,560459511002,2684454,1,0.916667
7660163,2021-09-27,560459511002,9032918,1,0.950000
7660164,2021-09-29,560459511002,27300245,1,0.983333
7660165,2021-09-29,560459511002,29254640,1,0.983333


# DB5: CBG to CBG
### Time-sensitive feature: count

In [9]:
db5_dir = os.path.join(raw_dir, 'hurricane_ida/unzip/Ida_CBG_CBG')
db5 = data_agg(db5_dir)
db5['time'] = time_normalization(db5['date'])

100%|██████████| 60/60 [00:14<00:00,  4.23it/s]


In [10]:
db5.to_csv(os.path.join(processed_dir, 'Ida_CBG_CBG.csv'), index=False)

In [11]:
db5

,date,begin_CBG,end_CBG,count,time
0,2021-08-01,220019601001,220019601001,30,0.0
1,2021-08-01,220019601001,220019601002,4,0.0
2,2021-08-01,220019601001,220019601003,5,0.0
3,2021-08-01,220019601001,220019601004,20,0.0
4,2021-08-01,220019601001,220019601005,14,0.0
...,...,...,...,...,...
9700305,2021-09-30,221259518004,221259517021,2,1.0
9700306,2021-09-30,221259518004,221259518001,10,1.0
9700307,2021-09-30,221259518004,221259518002,28,1.0
9700308,2021-09-30,221259518004,221259518003,62,1.0


# DB6: POI to POI
### Time-sensitive feature: count

In [12]:
db6_dir = os.path.join(raw_dir, 'hurricane_ida/unzip/Ida_POI_POI')
db6 = data_agg(db6_dir)
db6['time'] = time_normalization(db6['date'])

100%|██████████| 61/61 [00:07<00:00,  8.30it/s]


In [13]:
db6.to_csv(os.path.join(processed_dir, 'Ida_POI_POI.csv'), index=False)

In [14]:
db6

,date,begin_POI,end_POI,count,time
0,2021-08-01,2459140,2535374,22,0.0
1,2021-08-01,2459185,2462992,1,0.0
2,2021-08-01,2459185,2916770,1,0.0
3,2021-08-01,2459185,3111442,1,0.0
4,2021-08-01,2459185,3286294,1,0.0
...,...,...,...,...,...
3795736,2021-09-30,30178383,29294692,1,1.0
3795737,2021-09-30,30178444,30178444,1,1.0
3795738,2021-09-30,30178456,30178456,1,1.0
3795739,2021-09-30,30178474,9023131,1,1.0
